# kafkaReceiveDataPy
This notebook receives data from Kafka on the topic 'test', and stores it in the 'time_test' table of Cassandra (created by cassandra_init.script in startup_script.sh).

```
CREATE KEYSPACE test_time WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};

CREATE TABLE test_time.sent_received(
 time_sent TEXT,
 time_received TEXT,
PRIMARY KEY (time_sent)
);
```

A message that gives the current time is received every second. 

In [1]:
TOPIC = 'dmi'
TOPIC = 'imu'
TOPIC = 'gps'

## Add dependencies

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'
import time

## Load modules and start SparkContext
Note that SparkContext must be started to effectively load the package dependencies. Two cores are used, since one is needed for running the Kafka receiver.

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [4]:
import argparse
import csv
import time
import json

from utils import parse_line_into_schema

from fastavro import writer, reader, parse_schema
import avro_schemas

In [5]:
debug = True

In [6]:
# PARSE ARGS AND SET PATHS, TOPICS
KEYSPACE = 'raw_instrument'
DATA_OUT_DIR = '/data/'
DMI_OUT_FILEPATH = DATA_OUT_DIR + 'dmi.csv'
IMU_OUT_FILEPATH = DATA_OUT_DIR + 'imu.csv'
LIDAR_OUT_FILEPATH = DATA_OUT_DIR + 'lidar.csv'
GPS_OUT_FILEPATH = DATA_OUT_DIR + 'gps.csv'
INSTRUMENTS = ['imu', 'dmi', 'lidar', 'gps']

if debug:
    instrument = TOPIC
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-i', '--instrument', help='which instrument data to save to disk {0}'.format(INSTRUMENTS))
    args = parser.parse_args()
    instrument = args.instrument.lower()
    
if 'dmi' in instrument:
    filepath = DMI_OUT_FILEPATH
    schema = avro_schemas.dmi
    field_names = [d['name'] for d in schema['fields']]
    TOPIC = 'dmi'
elif 'imu' in instrument:
    filepath = IMU_OUT_FILEPATH
    schema = avro_schemas.imu
    field_names = [d['name'] for d in schema['fields']]
    TOPIC = 'imu'
elif 'lidar' in instrument:
    filepath = LIDAR_OUT_FILEPATH
    schema = avro_schemas.lidar
    field_names = [d['name'] for d in schema['fields']]
    TOPIC = 'lidar'
elif 'gps' in instrument:
    filepath = GPS_OUT_FILEPATH
    schema = avro_schemas.gps
    field_names = [d['name'] for d in schema['fields']]
    TOPIC = 'gps'
else:
    print('ERROR: ARGUMENT {0} NOT IN INSTRUMENTS {1}'.format(args.instrument, INSTRUMENTS))
    assert False

parsed_schema = parse_schema(schema)


In [7]:
parse_schema(schema)

{'__fastavro_parsed': True,
 'fields': [{'name': 'coordinate_id', 'type': 'int'},
  {'name': 'gps_timestamp', 'type': 'string'},
  {'name': 'latitude', 'type': ['float', 'null']},
  {'name': 'lat_dir', 'type': 'string'},
  {'name': 'longitude', 'type': ['float', 'null']},
  {'name': 'lon_dir', 'type': 'string'},
  {'name': 'num_sats', 'type': ['int', 'null']},
  {'name': 'horizontal_dil', 'type': ['float', 'null']},
  {'name': 'altitude', 'type': ['float', 'null']},
  {'name': 'altitude_units', 'type': 'string'},
  {'name': 'geo_sep_units', 'type': 'string'},
  {'name': 'age_gps_data', 'type': 'string'},
  {'name': 'unused', 'type': 'string'}],
 'name': 'gps.avro.sidewalk_rig',
 'type': 'record'}

In [8]:
field_names

['coordinate_id',
 'gps_timestamp',
 'latitude',
 'lat_dir',
 'longitude',
 'lon_dir',
 'num_sats',
 'horizontal_dil',
 'altitude',
 'altitude_units',
 'geo_sep_units',
 'age_gps_data',
 'unused']

In [9]:
len(field_names)

13

## SaveToCassandra function
Takes a list of tuple (rows) and save to Cassandra 

In [10]:
def saveToCassandra(rows):
    if not rows.isEmpty(): 
        sqlContext.createDataFrame(rows).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table=TOPIC, keyspace=KEYSPACE)\
        .save()

## Create streaming task
* Receive data from Kafka 'test' topic every five seconds
* Get stream content, and add receiving time to each message
* Save each RDD in the DStream to Cassandra. Also print on screen

In [11]:
# row = '1556477374944823040,26,3821,3751,150,113'.split(',')
# record = parse_line_into_schema(row, schema)
# record

In [12]:
ssc = StreamingContext(sc, 3)
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {TOPIC: 1})
#data = kvs.map(lambda x: x[1])
data = kvs.map(lambda x: json.loads(x[1]))
# rows= data.map(lambda x:Row(time_sent=x,time_received=time.strftime("%Y-%m-%d %H:%M:%S")))
# rows = data.map(lambda x: x.split(',')).map(lambda x: Row(field_names, x))
# rows = data.map(lambda x: x.split(','))
rows = data.map(lambda x: x)
#rows = data.map(lambda x: parse_line_into_schema(x, schema))
#rows= data.map(lambda x:Row(x))

rows.foreachRDD(saveToCassandra)
rows.pprint()

## Start streaming

In [13]:
ssc.start()

-------------------------------------------
Time: 2019-04-29 13:49:12
-------------------------------------------



/home/guest/spark/python/pyspark/sql/session.py:340: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


-------------------------------------------
Time: 2019-04-29 13:49:15
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.4', u'longitude': u'-122.91594333333333', u'unused': u'', u'gps_timestamp': u'20:23:52.400000', u'age_gps_data': u'', u'num_sats': u'05', u'latitude': u'46.90925333333333', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556545672109842176}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.4', u'longitude': u'-122.91594333333333', u'unused': u'', u'gps_timestamp': u'20:23:52.600000', u'age_gps_data': u'', u'num_sats': u'05', u'latitude': u'46.90925333333333', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556545672311533824}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.4', u'longitude': u'-122.91594', u'unused': u'', u'gps_timestamp': u'20:23:52.800000', u'age_gps_data': u'', u'num_sats':

-------------------------------------------
Time: 2019-04-29 13:51:21
-------------------------------------------

-------------------------------------------
Time: 2019-04-29 13:51:24
-------------------------------------------

-------------------------------------------
Time: 2019-04-29 13:51:27
-------------------------------------------

-------------------------------------------
Time: 2019-04-29 13:51:30
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'44.8', u'longitude': u'-122.91586833333334', u'unused': u'', u'gps_timestamp': u'20:23:16', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909333333333336', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556545887079664896}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'44.8', u'longitude': u'-122.91586666666667', u'unused': u'', u'gps_timestamp': u'20:23:16.200000', u'age_gps_data': u'', u'num_s

-------------------------------------------
Time: 2019-04-29 13:51:39
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.0', u'longitude': u'-122.91584833333333', u'unused': u'', u'gps_timestamp': u'20:23:24.800000', u'age_gps_data': u'', u'num_sats': u'07', u'latitude': u'46.909346666666664', u'horizontal_dil': u'1.4', u'altitude_units': u'M', u'coordinate_id': 1556545895956732160}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.0', u'longitude': u'-122.91584833333333', u'unused': u'', u'gps_timestamp': u'20:23:25', u'age_gps_data': u'', u'num_sats': u'07', u'latitude': u'46.909346666666664', u'horizontal_dil': u'1.4', u'altitude_units': u'M', u'coordinate_id': 1556545896157725184}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.0', u'longitude': u'-122.91584666666667', u'unused': u'', u'gps_timestamp': u'20:23:25.200000', u'age_gps_data': u'', u'num_sa

...

-------------------------------------------
Time: 2019-04-29 14:13:15
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.1', u'longitude': u'-122.915025', u'unused': u'', u'gps_timestamp': u'20:29:31.800000', u'age_gps_data': u'', u'num_sats': u'08', u'latitude': u'46.90937666666667', u'horizontal_dil': u'1.1', u'altitude_units': u'M', u'coordinate_id': 1556547191813707008}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.1', u'longitude': u'-122.91502166666666', u'unused': u'', u'gps_timestamp': u'20:29:32', u'age_gps_data': u'', u'num_sats': u'08', u'latitude': u'46.909375', u'horizontal_dil': u'1.1', u'altitude_units': u'M', u'coordinate_id': 1556547192014534912}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.1', u'longitude': u'-122.91501666666667', u'unused': u'', u'gps_timestamp': u'20:29:32.200000', u'age_gps_data': u'', u'num_sats': u'08', u'lati

-------------------------------------------
Time: 2019-04-29 14:13:24
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.4', u'longitude': u'-122.91492666666667', u'unused': u'', u'gps_timestamp': u'20:29:40.800000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.90933666666667', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547200858850048}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.4', u'longitude': u'-122.914925', u'unused': u'', u'gps_timestamp': u'20:29:41', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909335', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547201060239104}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.4', u'longitude': u'-122.91492333333333', u'unused': u'', u'gps_timestamp': u'20:29:41.200000', u'age_gps_data': u'', u'num_sats': u'06', u'lati

-------------------------------------------
Time: 2019-04-29 14:13:33
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.3', u'longitude': u'-122.91486333333333', u'unused': u'', u'gps_timestamp': u'20:29:49.800000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909301666666664', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547209905352960}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.3', u'longitude': u'-122.91486333333333', u'unused': u'', u'gps_timestamp': u'20:29:50', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909301666666664', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547210107333888}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.3', u'longitude': u'-122.91486166666667', u'unused': u'', u'gps_timestamp': u'20:29:50.200000', u'age_gps_data': u'', u'num_sa

-------------------------------------------
Time: 2019-04-29 14:13:42
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.5', u'longitude': u'-122.91481166666667', u'unused': u'', u'gps_timestamp': u'20:29:58.800000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909295', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547218950042112}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.5', u'longitude': u'-122.91481', u'unused': u'', u'gps_timestamp': u'20:29:59', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909295', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547219151042048}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'49.6', u'longitude': u'-122.91480666666666', u'unused': u'', u'gps_timestamp': u'20:29:59.200000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'

-------------------------------------------
Time: 2019-04-29 14:13:51
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.0', u'longitude': u'-122.91463166666666', u'unused': u'', u'gps_timestamp': u'20:30:07.800000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909275', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547228013452032}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.0', u'longitude': u'-122.91462833333334', u'unused': u'', u'gps_timestamp': u'20:30:08', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u'46.909275', u'horizontal_dil': u'1.5', u'altitude_units': u'M', u'coordinate_id': 1556547228214251008}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'50.0', u'longitude': u'-122.914625', u'unused': u'', u'gps_timestamp': u'20:30:08.200000', u'age_gps_data': u'', u'num_sats': u'06', u'latitude': u

## Stop streaming

In [14]:
ssc.stop(stopSparkContext=False,stopGraceFully=True)

-------------------------------------------
Time: 2019-04-29 13:47:45
-------------------------------------------
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.1', u'longitude': u'-122.915895', u'unused': u'', u'gps_timestamp': u'20:23:40.200000', u'age_gps_data': u'', u'num_sats': u'04', u'latitude': u'46.90936', u'horizontal_dil': u'4.5', u'altitude_units': u'M', u'coordinate_id': 1556545659839014144}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.1', u'longitude': u'-122.915895', u'unused': u'', u'gps_timestamp': u'20:23:40.400000', u'age_gps_data': u'', u'num_sats': u'04', u'latitude': u'46.90936', u'horizontal_dil': u'4.5', u'altitude_units': u'M', u'coordinate_id': 1556545660039872000}
{u'geo_sep_units': u'M', u'lon_dir': u'W', u'lat_dir': u'N', u'altitude': u'45.1', u'longitude': u'-122.915895', u'unused': u'', u'gps_timestamp': u'20:23:40.600000', u'age_gps_data': u'', u'num_sats': u'04', u'latitude': u'46.9093583

-------------------------------------------
Time: 2019-04-29 13:48:00
-------------------------------------------

-------------------------------------------
Time: 2019-04-29 13:48:05
-------------------------------------------



## Get Cassandra table content

In [15]:
TOPIC

'gps'

In [16]:
data=sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table=TOPIC, keyspace=KEYSPACE)\
    .load()
data.show()

+-------------------+------------+--------+--------------+-------------+---------------+--------------+-------+---------+-------+-----------+--------+------+
|      coordinate_id|age_gps_data|altitude|altitude_units|geo_sep_units|  gps_timestamp|horizontal_dil|lat_dir| latitude|lon_dir|  longitude|num_sats|unused|
+-------------------+------------+--------+--------------+-------------+---------------+--------------+-------+---------+-------+-----------+--------+------+
|1556545637068933120|            |    44.8|             M|            M|20:23:17.600000|           1.5|      N| 46.90933|      W| -122.91587|       6|      |
|1556545653989368064|            |    45.0|             M|            M|20:23:34.400000|           2.3|      N|46.909348|      W|-122.915855|       5|      |
|1556545636264638976|            |    44.8|             M|            M|20:23:16.800000|           1.5|      N| 46.90933|      W| -122.91586|       6|      |
|1556545639683782144|            |    45.0|         

## Get Cassandra table content using SQL

In [17]:
data.registerTempTable("TOPIC");
data.printSchema()
data=sqlContext.sql("select * from TOPIC")
data.show()

root
 |-- coordinate_id: long (nullable = true)
 |-- age_gps_data: string (nullable = true)
 |-- altitude: float (nullable = true)
 |-- altitude_units: string (nullable = true)
 |-- geo_sep_units: string (nullable = true)
 |-- gps_timestamp: string (nullable = true)
 |-- horizontal_dil: float (nullable = true)
 |-- lat_dir: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- lon_dir: string (nullable = true)
 |-- longitude: float (nullable = true)
 |-- num_sats: integer (nullable = true)
 |-- unused: string (nullable = true)

+-------------------+------------+--------+--------------+-------------+---------------+--------------+-------+---------+-------+-----------+--------+------+
|      coordinate_id|age_gps_data|altitude|altitude_units|geo_sep_units|  gps_timestamp|horizontal_dil|lat_dir| latitude|lon_dir|  longitude|num_sats|unused|
+-------------------+------------+--------+--------------+-------------+---------------+--------------+-------+---------+-------+------

In [18]:
data.count()

182